<span style="color: red; font-family: Calibri Light;">
  <h1><b>Topic Modelling with LDA: Evaluation</b></h1>
    <p style = "color: black"> Using LDA model on pre-processed data. Stop word removal during pre-processing is limited to common english stop words, top 100 most common words, and words occuring less than 10 times.<br>Any further stop word removal will be done when creating bag of words with the gensim library 
</span>

<span style="color: red; font-family: Calibri Light;">
  <h2><b>I. Setting Up Environment</b></h2>
</span>

In [1]:
#data transformation libraries
import pandas as pd
from pandas import option_context
import numpy as np
import ast
import csv

#NLP specific libraries
from gensim.models import Word2Vec

#topic modelling libraries
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel, LdaMulticore


#for visualization
import matplotlib.pyplot as plt
import seaborn as sns
from prettytable import PrettyTable
import pyLDAvis
import pyLDAvis.gensim_models

from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator

#others
import time
import os
import random
from glob import glob
import pickle


In [2]:
#set seed so that code output is deterministic
random.seed(200)  # Set the seed for Python's random module
np.random.seed(200)  # Set the seed for NumPy's random module

<span style="color: red; font-family: Calibri Light;">
  <h2><b>II. Import Data</b></h2>
</span>

<span style="color: red; font-family: Calibri Light;">
  <h2><b>a. Import preprocessed data</b></h2>
</span>

In [3]:
#import training_data

def list_converter(text):
    #to revert list->str conversion from pd.read_csv
    return ast.literal_eval(text)


data = pd.read_csv('../../../Data/lda_train.csv', converters ={'tokens':list_converter})
data = data.drop(columns = ['index'])
data.sort_values(by='date_created', inplace = True, ignore_index = True)
data.head()

,text_type,ID,date_created,year,long_text,clean_text,tokens,word_count
0,comment,c6d18gk,2012-09-25 07:57:13,2012,Yet i stared at the picture for a good 45 seco...,stare picture second miss,"[stare, picture, second, miss]",4
1,comment,c6d2fss,2012-09-25 09:13:23,2012,"[FYSR] = from your sister subreddit.\n\nIMO, i...",sister subreddit mildly interesting chance eve...,"[sister, subreddit, mildly, interesting, chanc...",18
2,comment,c6d46es,2012-09-25 12:32:08,2012,common give prince william harry a break he ju...,common prince harry break,"[common, prince, harry, break]",4
3,submission,1sur9h,2013-12-14 11:02:08,2013,Took this image of the Burj Khalifa from Souk ...,image burj khalifa souk bahar yesterday build ...,"[image, burj, khalifa, souk, bahar, yesterday,...",11
4,comment,ce1gf68,2013-12-14 12:07:24,2013,"Sorry pal, but you took an artistically not-so...",sorry impressive photo landmark resident karma...,"[sorry, impressive, photo, landmark, resident,...",8


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61376 entries, 0 to 61375
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   text_type     61376 non-null  object
 1   ID            61376 non-null  object
 2   date_created  61376 non-null  object
 3   year          61376 non-null  int64 
 4   long_text     61376 non-null  object
 5   clean_text    61376 non-null  object
 6   tokens        61376 non-null  object
 7   word_count    61376 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 3.7+ MB


<span style="color: red; font-family: Calibri Light;">
  <h2><b>b. Import Bag-of-Words</b></h2>
</span>

In [5]:
docs = data['tokens'].tolist()
# Create bigrams - code from gensim documentation page
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
            
dictionary = corpora.Dictionary(docs)

In [6]:
with open("../models/bow_corpus.pkl", "rb") as f:
    bow = pickle.load(f)

In [8]:
bow[0]

[(0, 1), (1, 1), (2, 1), (3, 1)]

<span style="color: red; font-family: Calibri Light;">
  <h2><b>c. Import Word2Vec Model</b></h2>
</span>

In [9]:
w2v_model = Word2Vec.load('../models/w2v_model_lda_bigrams.model')

<span style="color: red; font-family: Calibri Light;">
  <h2><b>III. LDA Model Visual Evaluation</b></h2>
</span>
<p>A review of the top 10 words in each topic to determine the following:
    <ul>
    <li>Do they make sense?</li>
    <li>can the topic be given a label?</li>
    <li>Using a sample of the submissions in the training data, determine if the topics are accurate
    </ul>

In [10]:
#functions for evaluating the models
#function to compute coherence, diversity and perplexity metrics
def eval_metrics (lda_model, docs, dictionary, num_topics, corpus, top_n = 10):
    #Compute c_v score
    c_v = CoherenceModel(model=lda_model, texts=docs, dictionary=dictionary, coherence='c_v')
    cv_lda = c_v.get_coherence()
    
    # Compute u_mass score
    u_mass = CoherenceModel(model=lda_model, texts=docs, dictionary=dictionary, coherence='u_mass')
    umass_lda = u_mass.get_coherence()
    
    # Compute c_npmi score
    c_npmi = CoherenceModel(model=lda_model, texts=docs, dictionary=dictionary, coherence='c_npmi')
    cnpmi_lda = c_npmi.get_coherence()
    
    # Compute perplexity
    perplexity = lda_model.log_perplexity(corpus)
    
    # Compute topic diversity
    top_words = [word for topic_id in range(num_topics) for word, _ in lda_model.show_topic(topic_id, topn=top_n)]
    diversity = len(set(top_words)) / (num_topics * top_n)
    
    print(f"For {num_topics} topics:\nCoherence(c_v) = {cv_lda},\nCoherence(c_npmi) = {cnpmi_lda},\nCoherence(u_mass) = {umass_lda},\nPerplexity = {perplexity},\nTopic Diversity = {diversity}\n")
    
    #return cv_lda, umass_lda, cnpmi_lda, perplexity, diversity

#function to check average word similarities for the topics 
def average_similarity(lda_model, num_topics, w2v_model, top_n=10):
    #extract top 10 words for each topic
    top_words_per_topic =[] 
    for topic_id in range(num_topics):
        top_words = lda_model.show_topic(topic_id, topn=top_n)
        top_words = [word for word, _ in top_words]
        top_words_per_topic.append(top_words)
        
    # 2. Compute pairwise similarities for each topic
    average_similarities = []
    for top_words in top_words_per_topic:
        total_similarity = 0
        count = 0
        for i in range(len(top_words)):
            for j in range(i+1, len(top_words)):  # Compare each word with the words after it
                if top_words[i] in w2v_model.wv and top_words[j] in w2v_model.wv:
                    similarity = w2v_model.wv.similarity(top_words[i], top_words[j])
                    total_similarity += similarity
                    count += 1
        average_similarity = total_similarity / count if count != 0 else 0
        average_similarities.append(average_similarity)
    #print average similarities for each topic
    for idx, avg_sim in enumerate(average_similarities):
        print(f"Average similarity for topic {idx}: {avg_sim:.4f}")
        
    #return average_similarities

<span style="color: red; font-family: Calibri Light;">
  <h2><b>a. model 1: 5 topics</b></h2>
</span>

In [11]:
model_1 = LdaModel.load('../models/model_1_5tpcs/lda_model_1_5tpcs')

In [12]:
#quantitative evaluation
eval_metrics (lda_model = model_1,docs = docs,
              dictionary = dictionary, num_topics = model_1.num_topics, 
               corpus = bow, top_n = 10)

average_similarity(lda_model = model_1, num_topics = model_1.num_topics, 
                   w2v_model = w2v_model, top_n=10)




For 5 topics:
Coherence(c_v) = 0.6049680135364921,
Coherence(c_npmi) = 0.01809764921147028,
Coherence(u_mass) = -3.6348576844708056,
Perplexity = -8.133242504850717,
Topic Diversity = 1.0

Average similarity for topic 0: 0.4982
Average similarity for topic 1: 0.4522
Average similarity for topic 2: 0.2859
Average similarity for topic 3: 0.4426
Average similarity for topic 4: 0.5138


In [12]:
model_1.print_topics(num_topics = -1)

[(0,
  '0.016*"report" + 0.012*"area" + 0.011*"car" + 0.011*"close" + 0.010*"fast" + 0.009*"speed" + 0.008*"metro" + 0.008*"traffic" + 0.008*"park" + 0.008*"turn"'),
 (1,
  '0.010*"bank" + 0.009*"rent" + 0.007*"sell" + 0.007*"send" + 0.007*"week" + 0.007*"property" + 0.007*"cheap" + 0.006*"order" + 0.006*"option" + 0.006*"charge"'),
 (2,
  '0.010*"mall" + 0.010*"walk" + 0.008*"reddit" + 0.007*"video" + 0.007*"stuff" + 0.006*"outside" + 0.006*"night" + 0.006*"tip" + 0.006*"visit" + 0.006*"social"'),
 (3,
  '0.007*"kid" + 0.006*"care" + 0.006*"hard" + 0.006*"speak" + 0.006*"sorry" + 0.006*"culture" + 0.006*"learn" + 0.006*"situation" + 0.006*"school" + 0.005*"believe"'),
 (4,
  '0.007*"passport" + 0.006*"allow" + 0.006*"government" + 0.006*"apply" + 0.006*"travel" + 0.005*"base" + 0.005*"rule" + 0.005*"employee" + 0.005*"india" + 0.005*"job"')]

In [14]:
#dataset of topcis and topic representation
num_topics = model_1.num_topics

topics_words = []

for topic in range(num_topics):
    topic_words = model_1.show_topic(topic, topn = 10)
    words = [word[0] for word in topic_words]
    topics_words.append({"topic": topic, "words": words})
    

#create a dataframe
model_1_topics_df = pd.DataFrame(topics_words)
model_1_topics_df.set_index('topic', inplace =True)

with option_context('display.max_colwidth', None):
    display(model_1_topics_df)
    


,words
topic,
0,"[report, area, car, close, fast, speed, metro, traffic, park, turn]"
1,"[bank, rent, sell, send, week, property, cheap, order, option, charge]"
2,"[mall, walk, reddit, video, stuff, outside, night, tip, visit, social]"
3,"[kid, care, hard, speak, sorry, culture, learn, situation, school, believe]"
4,"[passport, allow, government, apply, travel, base, rule, employee, india, job]"


<ul>
    <li>topic 0: <em>'car', 'fast', 'speed', 'park', 'turn', 'traffic'</em> --> <em><b>urban_mobility.traffic_regulations</b></em>.
    That means of the top 10 words, 6 relate to the same topic.</li><br>
    <li>topic 1: <em> 'rent', 'property','sell' </em> --> <em><b>real_estate_housing</b></em>. However, the terms <em> 'sell', 'cheap', 'order'</em> --> <em><b>shopping_services</b></em>. Also, <em> 'bank', 'charge', 'send'</em> --><em><b>banking_financial_services</b></em> This topic contains three sub-topics.</li><br>
    <li>topic 2: <em>'mall', 'walk', 'outside', 'night', 'visit',</em>label --> <em><b>recreation</b></em>, but also contains terms (<em>'reddit', 'video', 'social'</em>) related to <em><b>communication_social_media</b></em>.</li><br>
    <li>topic 3: <em>'kid','care', 'hard',' learn', 'school'</em>, label --> <em><b>education</b></em></li><br>
    <li>topic 4: <em>'passport', 'travel' </em>--> <em><b>immigration_travel</b></em>, but the topic also contains <em>'apply', 'base', 'employee', 'job'</em> --> <em><b>employment_opportunities</b></em></li>
</ul>
<p> Of the five topics extracted, only 1 has up to 5 coherent words that indicate a topic of interest i.e topic 0 for <em><b>urban_mobility</b></em>. Topics 1 and 4 have sub topics in them that are not related to each other.This model is likely not the best fit for our data.</p>

In [17]:
#attach topic label to topic terms

topic_label ={
    0: "urban_mobility.traffic_regulations",
    1: "accommodation/shopping_purchases/financial_services",
    2: "recreation/social_media",
    3: "education",
    4: "employment_opportunities/immigration_travel",
}

model_1_topics_df['label'] = model_1_topics_df.index.map(topic_label)

with option_context('display.max_colwidth', None):
    display(model_1_topics_df)

,words,label
topic,,
0,"[report, area, car, close, fast, speed, metro, traffic, park, turn]",urban_mobility.traffic_regulations
1,"[bank, rent, sell, send, week, property, cheap, order, option, charge]",accommodation/shopping_purchases/financial_services
2,"[mall, walk, reddit, video, stuff, outside, night, tip, visit, social]",recreation/social_media
3,"[kid, care, hard, speak, sorry, culture, learn, situation, school, believe]",education
4,"[passport, allow, government, apply, travel, base, rule, employee, india, job]",employment_opportunities/immigration_travel


In [55]:
#include column for most probable topic for each entry

model_1_data = data.copy() #copy of the trainin data
top_topic_per_document = [] #list to hold the topic with the highest probability for each row of texts in the data
top_topic2_per_document = [] #list to hold the topic with the second highest probability for each row of texts in the data
# Create a mapping from topics to labels
topic_to_label = model_1_topics_df.set_index('topic')['label'].to_dict()


for doc in bow:
    topics = model_1.get_document_topics(doc, minimum_probability = 0)
    top_topic = sorted(topics, key=lambda x: x[1], reverse = True)[0][0]
    top_topic2 = sorted(topics, key=lambda x: x[1], reverse = True)[1][0]
    top_topic_per_document.append(top_topic)
    top_topic2_per_document.append(top_topic2)
    
#add column to data dataframe for the selected topic
model_1_data['top_topic'] = top_topic_per_document
# Map the top_topic to label
model_1_data['label'] = model_1_data['top_topic'].map(topic_to_label)

#add column to data dataframe for the selected topic
model_1_data['topic_2'] = top_topic2_per_document
# Map the top_topic to label
model_1_data['label_2'] = model_1_data['topic_2'].map(topic_to_label)


In [ ]:
#visually evaluate a small subset of submissions
sample = model_1_data[model_1_data.text_type == 'submission'].sample(n = 10, random_state = 42)

with option_context('display.max_colwidth', None):
    display(sample[['date_created', 'long_text', "top_topic",'label', "topic_2", 'label_2']])

---

<span style="color: red; font-family: Calibri Light;">
  <h2><b>a. model 2: 10 topics</b></h2>
</span>

In [18]:
model_2 = LdaModel.load('../models/model_2_10tpcs/lda_model_2_10tpcs')

In [28]:
#quantitative evaluation
eval_metrics (lda_model = model_2,docs = docs,
              dictionary = dictionary, num_topics = model_2.num_topics, 
               corpus = bow, top_n = 10)

average_similarity(lda_model = model_2, num_topics = model_2.num_topics, 
                   w2v_model = w2v_model, top_n=10)




For 10 topics:
Coherence(c_v) = 0.4895138013398749,
Coherence(c_npmi) = -0.02281372631638639,
Coherence(u_mass) = -4.798116025994018,
Perplexity = -8.206242643921952,
Topic Diversity = 1.0

Average similarity for topic 0: 0.7092
Average similarity for topic 1: 0.5578
Average similarity for topic 2: 0.4880
Average similarity for topic 3: 0.5324
Average similarity for topic 4: 0.5290
Average similarity for topic 5: 0.6461
Average similarity for topic 6: 0.5922
Average similarity for topic 7: 0.3237
Average similarity for topic 8: 0.4213
Average similarity for topic 9: 0.4569


In [19]:
display(model_2.print_topics(num_topics = -1))

[(0,
  '0.023*"fast" + 0.021*"car" + 0.021*"speed" + 0.020*"traffic" + 0.017*"limit" + 0.015*"license" + 0.012*"vehicle" + 0.011*"slow" + 0.011*"ticket" + 0.011*"plate"'),
 (1,
  '0.019*"bank" + 0.017*"rent" + 0.013*"property" + 0.011*"send" + 0.011*"plan" + 0.011*"credit" + 0.011*"account" + 0.010*"sell" + 0.010*"market" + 0.010*"charge"'),
 (2,
  '0.010*"area" + 0.010*"hour" + 0.009*"week" + 0.009*"close" + 0.008*"open" + 0.008*"walk" + 0.008*"visit" + 0.007*"mall" + 0.007*"away" + 0.006*"usually"'),
 (3,
  '0.030*"speak" + 0.029*"learn" + 0.028*"school" + 0.022*"word" + 0.019*"arabic" + 0.018*"english" + 0.017*"play" + 0.016*"language" + 0.015*"rich" + 0.013*"fuck"'),
 (4,
  '0.025*"test" + 0.020*"tip" + 0.017*"covid" + 0.015*"daily" + 0.014*"health" + 0.012*"thread" + 0.010*"positive" + 0.009*"medical" + 0.009*"pregnant" + 0.009*"june"'),
 (5,
  '0.018*"report" + 0.014*"reddit" + 0.012*"message" + 0.012*"website" + 0.011*"video" + 0.010*"detail" + 0.010*"google" + 0.010*"write" + 0

In [20]:
#dataset of topcis and topic representation
num_topics = model_2.num_topics

topics_words = []

for topic in range(num_topics):
    topic_words = model_2.show_topic(topic, topn = 10)
    words = [word[0] for word in topic_words]
    topics_words.append({"topic": topic, "words": words})
    

#create a dataframe
model_2_topics_df = pd.DataFrame(topics_words)
model_2_topics_df.set_index('topic', inplace =True)

with option_context('display.max_colwidth', None):
    display(model_2_topics_df)
    


,words
topic,
0,"[fast, car, speed, traffic, limit, license, vehicle, slow, ticket, plate]"
1,"[bank, rent, property, send, plan, credit, account, sell, market, charge]"
2,"[area, hour, week, close, open, walk, visit, mall, away, usually]"
3,"[speak, learn, school, word, arabic, english, play, language, rich, fuck]"
4,"[test, tip, covid, daily, health, thread, positive, medical, pregnant, june]"
5,"[report, reddit, message, website, video, detail, google, write, link, news]"
6,"[cheap, order, restaurant, delivery, expensive, quality, contract, bill, store, extra]"
7,"[middle, agent, class, crime, east, low, saudi, project, mask, power]"
8,"[kid, movie, parent, child, wife, muslim, wear, hate, religion, accident]"


<ul>
    <li>topic 0: The top 10 words are all representative terms --> <em><b>urban_mobility.traffic_regulations</b></em>.</li><br>
    <li>topic 1: <em> property', 'rent', 'sell'</em> -->  <em><b>real_estate_housing</b></em>. However, the terms <em> 'sell', 'market', 'charge',</em> --> <em><b>shopping and purchases</b></em>.Also, <em>'bank', 'credit', 'account', 'charge','send'</em> --> <em><b> banking/financial_services</b></em>.This topic contains three themes.</li><br>
    <li>topic 2: <em>'mall', 'walk', 'visit'</em>might be said to indicate <em><b>recreation</b></em>, but its a bit of a stretch</li><br>
    <li>topic 3: <em> 'speak', 'learn', 'school', 'word', 'language'</em> -->  <em><b>educational_services</b></em>. However, the terms <em> 'arabic', 'english', 'language'</em> --> <em><b>inter-cultural_relationships</b></em>.</li><br>
    <li>topic 4: <em>'test', 'covid', 'health', 'positive', 'medical', 'pregnant'</em>--><em><b>health_services</b></em></li><br>
    <li>topic 5: label --><em><b>communications_social_media</b></em></li><br>
    <li> topic 6: terms belonging to two themes, labels --> <em><b>shopping/dining_experience</b></em>.</li><br>
    <li>topic 7: contains a jumble of words that don't fit together to specify one coherent topic,  or a topic of interest</li><br>
    <li>topic 8: <em>'kid', 'parent','child', 'wife'</em>--> <em><b>social_relationships</b></em>, <em>'muslim','religion', 'wear'</em>--> <em><b>religion</b></em></li><br>
    <li>topic 9: contains a jumble of words that don't fit together to specify one coherent topic, or a topic of interest</li><br>
</ul>
<p> Of the 10 topics extracted, three have more than 5 terms that belong to a coherent topic, while two contain sub-topics. This model presents better granular topis than model 1, but is still unable to fully split different concepts into separate topics as seen in topics 1,2,3.</p>

In [83]:
#attach topic label to topic terms

topic_label ={
    0: "urban_mobility",
    1: "financial_services",
    2: "undefined",
    3: "educational_services",
    4: "health_services",
    5: "undefined",
    6: "food/dining_experience",
    7: "undefined",
    8: "social_relationships",
    9: "undefined"
}

model_2_topics_df['label'] = model_2_topics_df['topic'].map(topic_label)

with option_context('display.max_colwidth', None):
    display(model_2_topics_df)

,topic,words,label
0,0,"[fast, car, speed, traffic, limit, license, vehicle, slow, ticket, plate]",urban_mobility
1,1,"[bank, rent, property, send, plan, credit, account, sell, market, charge]",financial_services
2,2,"[area, hour, week, close, open, walk, visit, mall, away, usually]",undefined
3,3,"[speak, learn, school, word, arabic, english, play, language, rich, fuck]",educational_services
4,4,"[test, tip, covid, daily, health, thread, positive, medical, pregnant, june]",health_services
5,5,"[report, reddit, message, website, video, detail, google, write, link, news]",undefined
6,6,"[cheap, order, restaurant, delivery, expensive, quality, contract, bill, store, extra]",food/dining_experience
7,7,"[middle, agent, class, crime, east, low, saudi, project, mask, power]",undefined
8,8,"[kid, movie, parent, child, wife, muslim, wear, hate, religion, accident]",social_relationships
9,9,"[care, consider, situation, passport, culture, hard, mention, matter, arab, believe]",undefined


In [69]:
#include column for most probable topic for each entry

model_2_data = data.copy() #copy of the trainin data
top_topic_per_document = [] #list to hold the topic with the highest probability for each row of texts in the data
top_topic2_per_document = [] #list to hold the topic with the second highest probability for each row of texts in the data
# Create a mapping from topics to labels
topic_to_label = model_2_topics_df.set_index('topic')['label'].to_dict()


for doc in bow:
    topics = model_2.get_document_topics(doc, minimum_probability = 0)
    top_topic = sorted(topics, key=lambda x: x[1], reverse = True)[0][0]
    top_topic2 = sorted(topics, key=lambda x: x[1], reverse = True)[1][0]
    top_topic_per_document.append(top_topic)
    top_topic2_per_document.append(top_topic2)
    
#add column to data dataframe for the selected topic
model_2_data['top_topic'] = top_topic_per_document
# Map the top_topic to label
model_2_data['label'] = model_2_data['top_topic'].map(topic_to_label)

#add column to data dataframe for the selected topic
model_2_data['topic_2'] = top_topic2_per_document
# Map the top_topic to label
model_2_data['label_2'] = model_2_data['topic_2'].map(topic_to_label)


In [70]:
#visually evaluate a small subset of submissions
sample = model_2_data[model_2_data.text_type == 'submission'].sample(n = 20, random_state = 42)

with option_context('display.max_colwidth', None):
    display(sample[['date_created', 'long_text', "top_topic",'label', "topic_2", 'label_2']])

,date_created,long_text,top_topic,label,topic_2,label_2
38303,2022-07-17 08:23:29,"Anybody else been forced to start a business after losing a job, and wake up every morning wishing you never had to open your eyes? Every day is a minor heart attack asking yourself, ""can I do this ethically?"" All you see is people conning each other, and all you hear is, ""this is how you have to play the game"" I've been conned by people who I thought were friends. And then been in business with another ""friend"" with questionable business practices. I'm not suicidal, but I do keep wishing this would all end.",2,undefined,8,social_relationships
59518,2023-06-18 20:08:38,"require some suggestions - banking / accounts / loans Hi All,\n\nNeed some suggestions or opinions and better personal experiences.\n\nI have been banking and have account and credit cards with a bank here for almost a decade. It's a conventional bank, but sooner or later I am going to go for a home loan - for which I would prefer an Islamic bank. Should I open some account with an Islamic bank to have some relationship over period of time OR it doesn't matter - when the need to loan arises I can go for looking around for loan and based on my AECB reports bank would provide rates - so effectively meaning that longer or shorter relationship wouldn't mean anything it just that transaction and the score at the point of time which would matter?",1,financial_services,9,undefined
3608,2020-02-10 10:02:39,Current Global Economic Situation,9,undefined,7,undefined
60089,2023-06-19 20:13:26,I need to get a broken hard disk repaired. I’ve got a ton of data on it with no backup. Any idea if there are any reliable places in Dubai or Sharjah for data recovery ? Tried searching online and didn’t find anything convincing.,1,financial_services,2,undefined
61240,2023-06-21 17:53:14,"What to do when the neighbour parks like this? Hello Dubai community!\n\nGuys it is getting out of hands, I never met this guy, but he keeps on parking like this recently. \n\nAny advice from professional residents how to deal with this?",9,undefined,2,undefined
6118,2020-05-11 11:11:08,Any news on gyms reopening? Does anybody have an insider scoop on when they're reopening?\n\nNewspaper people if you are reading you can chime in. In the form of a news article in 2 days or something.,5,undefined,9,undefined
36450,2022-06-06 21:33:34,Experience with Techem ACs I’m planning to move in a building which has Techem as their AC providers. Can someone share their experience with the bills? I am hearing they are quite high.,6,food/dining_experience,2,undefined
9116,2020-06-16 13:35:29,"Entry permit validity Hey fellas, just came to know that the entry permit validity for residents stuck abroad is 21 days. It’s there on their website and the call centre also confirmed it. But given that many airports are still not opened up to international travel and fewer flights operating, what happens if one is not able to make it into UAE before the 21 day limit? Any input is appreciated. Thanks.",1,financial_services,9,undefined
52698,2023-04-19 08:54:12,Where can I find reasonably-priced jewelry and gold in Dubai? Looking to get my mam a gift Where can I find reasonably-priced jewelry and gold in Dubai? Looking to get my mam a gift,0,urban_mobility,3,educational_services
42845,2022-09-22 17:29:32,"Any CUD current or former students here? If so, any advice, stories or highlights for a soon to be student at the Uni? General Uni advice appreciated as well. \nI’m hopefully gonna be attending the spring semester next year and studying Computer Engineering but been getting cold feet since i don’t really know anyone there and don’t know what to expect.\n\n\nAnd while i’ve heard plenty of praise for this Uni i’ve also heard rumors of there being a whole social status hierarchy deal going on which has me nervous-ish.\n\nEdit: Just so yk i already heard the song and dance of Unis here not being great but i don’t have a choice but to study in

<span style="color: red; font-family: Calibri Light;">
  <h2><b>c. model 2: 15 topics</b></h2>
</span>

In [76]:
model_3 = LdaModel.load('../models/model_3_15tpcs/lda_model_3_15tpcs')

In [77]:
#quantitative evaluation
eval_metrics (lda_model = model_3,docs = docs,
              dictionary = dictionary, num_topics = model_3.num_topics, 
               corpus = bow, top_n = 10)

average_similarity(lda_model = model_3, num_topics = model_3.num_topics, 
                   w2v_model = w2v_model, top_n=10)




For 15 topics:
Coherence(c_v) = 0.46347251677274065,
Coherence(c_npmi) = -0.026488737095068004,
Coherence(u_mass) = -5.084119863191523,
Perplexity = -8.920892722416768,
Topic Diversity = 0.9933333333333333

Average similarity for topic 0: 0.3907
Average similarity for topic 1: 0.5440
Average similarity for topic 2: 0.7234
Average similarity for topic 3: 0.1976
Average similarity for topic 4: 0.3399
Average similarity for topic 5: 0.6650
Average similarity for topic 6: 0.4382
Average similarity for topic 7: 0.6379
Average similarity for topic 8: 0.5084
Average similarity for topic 9: 0.4833
Average similarity for topic 10: 0.4693
Average similarity for topic 11: 0.2890
Average similarity for topic 12: 0.4452
Average similarity for topic 13: 0.4227
Average similarity for topic 14: 0.5073


In [78]:
display(model_3.print_topics(num_topics = -1))

[(0,
  '0.028*"area" + 0.027*"mall" + 0.027*"walk" + 0.020*"wear" + 0.020*"visit" + 0.020*"dhabi" + 0.017*"public" + 0.016*"social" + 0.015*"rule" + 0.015*"hotel"'),
 (1,
  '0.020*"rent" + 0.017*"sell" + 0.015*"property" + 0.015*"cheap" + 0.015*"order" + 0.015*"option" + 0.014*"restaurant" + 0.013*"charge" + 0.013*"delivery" + 0.013*"credit"'),
 (2,
  '0.027*"fast" + 0.025*"second" + 0.025*"speed" + 0.023*"traffic" + 0.022*"line" + 0.020*"limit" + 0.018*"light" + 0.016*"pass" + 0.013*"slow" + 0.012*"park"'),
 (3,
  '0.047*"report" + 0.027*"reddit" + 0.026*"stand" + 0.023*"middle" + 0.022*"door" + 0.020*"tip" + 0.015*"rental" + 0.013*"east" + 0.010*"brother" + 0.010*"dear"'),
 (4,
  '0.021*"sense" + 0.021*"soon" + 0.016*"common" + 0.013*"possible" + 0.011*"amazing" + 0.010*"expect" + 0.009*"demand" + 0.008*"app" + 0.008*"culture" + 0.007*"past"'),
 (5,
  '0.018*"bank" + 0.017*"send" + 0.015*"phone" + 0.013*"account" + 0.013*"online" + 0.012*"website" + 0.011*"contact" + 0.011*"email" + 

In [93]:
#dataset of topcis and topic representation
num_topics = model_3.num_topics

topics_words = []

for topic in range(num_topics):
    topic_words = model_3.show_topic(topic, topn = 10)
    words = [word[0] for word in topic_words]
    topics_words.append({"topic": topic, "words": words})
    

#create a dataframe
model_3_topics_df = pd.DataFrame(topics_words)
model_3_topics_df.set_index('topic', inplace =True)

with option_context('display.max_colwidth', None):
    display(model_3_topics_df)
    


,words
topic,
0,"[area, mall, walk, wear, visit, dhabi, public, social, rule, hotel]"
1,"[rent, sell, property, cheap, order, option, restaurant, charge, delivery, credit]"
2,"[fast, second, speed, traffic, line, limit, light, pass, slow, park]"
3,"[report, reddit, stand, middle, door, tip, rental, east, brother, dear]"
4,"[sense, soon, common, possible, amazing, expect, demand, app, culture, past]"
5,"[bank, send, phone, account, online, website, contact, email, process, book]"
6,"[hour, kid, week, school, hand, seat, parent, clean, early, travel]"
7,"[arab, government, indian, culture, india, true, expat, western, nationality, racism]"
8,"[insurance, test, car, license, cover, vehicle, tenant, expense, plate, fail]"


<ul>
    <li>topic 0: contains terms that belong to different themes,and cannot fit into one main coherent them</li><br>
    <li>topic 1: <em>'rent', 'sell', 'property', 'cheap'</em>--><em><b>real_estate_housing</b></em>. But also, <em>cheap', 'order', 'restaurant', 'delivery'</em>--><em><b>food_dining_services</b></em>, but also, <em>'order', 'option', 'charge', 'credit'</em>--><em><b>shopping_services</b></em></li><br>
    <li>topic 2: <em>'fast', 'speed', 'traffic', 'limit', 'light', 'pass', 'slow'</em>--><em><b>urban_mobility.traffic_regulations</b></em></li><br>
    <li>topic 3: non_context</li><br>
    <li>topic 4: non_context </li><br>
    <li>topic 5: contains terms that belong to different themes,and cannot fit into one main coherent theme</li><br>
    <li>topic 6: jumbo of terms that fit into different categories</li><br>
    <li>topic 7: <em>'arab', 'indian', 'india', 'expat', 'western', 'nationality'</em>--><em><b>international_communities</b></em></li><br>
    <li>topic 8:<em> 'insurance', 'test', 'car', 'license', 'cover', 'vehicle', 'plate'</em>--><em><b>urban_mobility.car_ownership</b></em></li><br>
    <li>topic 9: <em> 'muslim', 'religion', 'pray'</em>--><em><b>religious practices</b></em> but also contains other terms that belong to other themes.</li><br>
    <li>topic 10: contains terms that belong to different themes,and cannot fit into one main coherent theme</li><br>
    <li>topic 11: contains terms that belong to different themes,and cannot fit into one main coherent theme</li><br>
    <li>topic 12: contains terms that belong to different themes,and cannot fit into one main coherent theme</li><br>
    <li>topic 13:<em>'date', 'watch', 'movie', 'drink'</em>--><em><b>social_activities</b></em>, but also includes other terms that do not fit</li><br>
    <li>topic 14: contains terms that belong to different themes,and cannot fit into one main coherent theme</li>
</ul>

In [92]:
#attach topic label to topic terms

topic_label ={
    0: "undefined",
    1: "real_estate_housing",
    2: "urban_mobility.traffic_regulations",
    3: "non_context",
    4: "non_context",
    5: "undefined",
    6: "undefined",
    7: "international_communities",
    8: "urban_mobility.car_ownership",
    9: "religious_practices",
    10: "undefined",
    11: "undefined",
    12: "undefined",
    13: "social_activities",
    14: "undefined"
}

model_3_topics_df['label'] = model_3_topics_df['topic'].map(topic_label)

with option_context('display.max_colwidth', None):
    display(model_3_topics_df)

,topic,words,label
0,0,"[area, mall, walk, wear, visit, dhabi, public, social, rule, hotel]",undefined
1,1,"[rent, sell, property, cheap, order, option, restaurant, charge, delivery, credit]",real_estate_housing
2,2,"[fast, second, speed, traffic, line, limit, light, pass, slow, park]",urban_mobility.traffic_regulations
3,3,"[report, reddit, stand, middle, door, tip, rental, east, brother, dear]",non_context
4,4,"[sense, soon, common, possible, amazing, expect, demand, app, culture, past]",non_context
5,5,"[bank, send, phone, account, online, website, contact, email, process, book]",undefined
6,6,"[hour, kid, week, school, hand, seat, parent, clean, early, travel]",undefined
7,7,"[arab, government, indian, culture, india, true, expat, western, nationality, racism]",international_communities
8,8,"[insurance, test, car, license, cover, vehicle, tenant, expense, plate, fail]",urban_mobility.car_ownership
9,9,"[muslim, religion, room, hate, steal, totally, ban, pregnant, flag, pray]",religious_practices


<span style="color: red; font-family: Calibri Light;">
  <h2><b>d. model 4: 20 topics</b></h2>
</span>

In [85]:
model_4 = LdaModel.load('../models/model_4_20tpcs/lda_model_4_20tpcs')

In [86]:
#quantitative evaluation
eval_metrics (lda_model = model_4,docs = docs,
              dictionary = dictionary, num_topics = model_4.num_topics, 
               corpus = bow, top_n = 10)

average_similarity(lda_model = model_4, num_topics = model_4.num_topics, 
                   w2v_model = w2v_model, top_n=10)




For 20 topics:
Coherence(c_v) = 0.45719477033031203,
Coherence(c_npmi) = -0.0282779422731149,
Coherence(u_mass) = -4.898497802637102,
Perplexity = -9.179299779882598,
Topic Diversity = 1.0

Average similarity for topic 0: 0.5438
Average similarity for topic 1: 0.5287
Average similarity for topic 2: 0.3807
Average similarity for topic 3: 0.5684
Average similarity for topic 4: 0.2829
Average similarity for topic 5: 0.4078
Average similarity for topic 6: 0.6791
Average similarity for topic 7: 0.5954
Average similarity for topic 8: 0.4531
Average similarity for topic 9: 0.2947
Average similarity for topic 10: 0.4836
Average similarity for topic 11: 0.4723
Average similarity for topic 12: 0.5367
Average similarity for topic 13: 0.3404
Average similarity for topic 14: 0.4035
Average similarity for topic 15: 0.5397
Average similarity for topic 16: 0.3451
Average similarity for topic 17: 0.5108
Average similarity for topic 18: 0.5223
Average similarity for topic 19: 0.6429


In [91]:
#dataset of topcis and topic representation
num_topics = model_4.num_topics

topics_words = []

for topic in range(num_topics):
    topic_words = model_4.show_topic(topic, topn = 10)
    words = [word[0] for word in topic_words]
    topics_words.append({"topic": topic, "words": words})
    

#create a dataframe
model_4_topics_df = pd.DataFrame(topics_words)
model_4_topics_df.set_index('topic', inplace =True)

with option_context('display.max_colwidth', None):
    display(model_4_topics_df)
    


,words
topic,
0,"[spend, compare, save, class, rate, earn, government, worker, increase, income]"
1,"[kid, hard, child, lose, accept, parent, care, exist, support, grow]"
2,"[play, owner, game, project, damn, lmao, build, fuel, fair, valid]"
3,"[apply, travel, passport, emirate, employee, require, process, book, hour, cancel]"
4,"[watch, arab, movie, station, scam, social, racist, clearly, medium, opinion]"
5,"[safe, airport, center, coffee, space, address, train, burj, accommodation, jumeirah]"
6,"[phone, send, reddit, message, video, link, website, answer, picture, block]"
7,"[speak, arabic, english, language, indian, word, learn, european, american, apple]"
8,"[sound, hand, wife, head, clean, taxi, crime, weird, paper, special]"


This model seems to have more merged topics
<ul>
    <li>topic 0: Label --><em><b>income</b></em>. Terms all revolve around income, except maybe for <em>'class', 'compare'</em>.</li><br>
    <li>topic 1: Label --> <em><b>social_relationships.childcare</b></em>. Terms mostly revolve around childcare, except maybe for <em>'hard', 'accept'</em>.</li><br>
    <li>topic 2: <em>'play', 'game'</em> can be labelled --> <em><b>entertainment</b></em> but none of the other terms fit in.</li><br>
    <li>topic 3: label--><em><b>immigration.travel_procedures</b></em>. All the terms fit except maybe for <em>employee.</em></li><br>
    <li>topic 4: terms cannot clearly be fit into one or related themes, or a theme we are interested in. label --> <em><b>undefined.</b></em></li><br>
    <li>topic 5: labels --> <em><b>infrastructure.landmarks/amenities</b></em>. Most terms fit except for 'accommodation' maybe.</li><br>
    <li>topic 6: labels --> <em><b>communication_media</b></em></li><br>
    <li>topic 7: labels --> <em><b>international_communities</b></em>. All terms seem to fit except 'apple'</li><br>
    <li>topic 8: a jumble of terms that dont share a coherent theme<li><br>
    <li>topic 9: a jumble of terms that dont share a coherent theme<li><br>
    <li>topic 10: label --> <em><b>food_dining_services</b></em></li><br>
    <li>topic 11: a mix of terms for <em><b>housing</b></em> and <em><b>infrastructure.amenities</b></em></li><br>
    <li>topic 12: terms cannot clearly be fit into one or related themes, or a theme we are interested in. label --> <em><b>undefined.</b></em></li><br>
    <li>topic 13: a jumble of terms that dont share a coherent theme<li><br>
    <li>topic 14: a jumble of terms that dont share a coherent theme<li><br>
    <li>topic 15: label --> <em><b>culture_norms</b></em></li><br>
    <li>topic 16: a jumble of terms that dont share a coherent theme<li><br>
    <li>topic 17: label --> <em><b>health.COVID</b></em>. These terms relate to behaviors exhibited during the COVID pandemic</li><br>
    <li>topic 18: some terms like <em>'bank', 'plan', 'account', 'credit'</em> are related to label --><em><b>financial_services</b></em></li><br>
    <li> topic 19: label --> <em><b>shopping_services</b></em></li><br>
        
</ul>

<span style="color: red; font-family: Calibri Light;">
  <h2><b>d. model 5: 25 topics</b></h2>
</span>

In [96]:
model_5 = LdaModel.load('../models/model_5_25tpcs/lda_model_5_25tpcs')

In [97]:
#quantitative evaluation
eval_metrics (lda_model = model_5,docs = docs,
              dictionary = dictionary, num_topics = model_5.num_topics, 
               corpus = bow, top_n = 10)

average_similarity(lda_model = model_5, num_topics = model_5.num_topics, 
                   w2v_model = w2v_model, top_n=10)




For 25 topics:
Coherence(c_v) = 0.4370927325782465,
Coherence(c_npmi) = -0.04470407911136721,
Coherence(u_mass) = -5.184369933326291,
Perplexity = -9.434405649658327,
Topic Diversity = 1.0

Average similarity for topic 0: 0.4606
Average similarity for topic 1: 0.4066
Average similarity for topic 2: 0.4484
Average similarity for topic 3: 0.5007
Average similarity for topic 4: 0.3327
Average similarity for topic 5: 0.3551
Average similarity for topic 6: 0.5517
Average similarity for topic 7: 0.3855
Average similarity for topic 8: 0.5423
Average similarity for topic 9: 0.5204
Average similarity for topic 10: 0.5186
Average similarity for topic 11: 0.4796
Average similarity for topic 12: 0.3581
Average similarity for topic 13: 0.3215
Average similarity for topic 14: 0.6221
Average similarity for topic 15: 0.6366
Average similarity for topic 16: 0.5926
Average similarity for topic 17: 0.5678
Average similarity for topic 18: 0.4921
Average similarity for topic 19: 0.3593
Average similarity f

In [99]:
display(model_5.print_topics(num_topics = -1))

[(0,
  '0.038*"report" + 0.028*"passport" + 0.026*"travel" + 0.020*"process" + 0.019*"emirate" + 0.018*"email" + 0.018*"contract" + 0.017*"wear" + 0.017*"cancel" + 0.016*"court"'),
 (1,
  '0.037*"owner" + 0.036*"head" + 0.035*"turn" + 0.030*"hotel" + 0.028*"room" + 0.021*"light" + 0.019*"lmao" + 0.017*"bother" + 0.017*"curious" + 0.015*"fresh"'),
 (2,
  '0.029*"hand" + 0.023*"face" + 0.022*"sense" + 0.019*"hold" + 0.017*"common" + 0.016*"literally" + 0.015*"public" + 0.015*"short" + 0.012*"stupid" + 0.012*"absolutely"'),
 (3,
  '0.038*"away" + 0.037*"date" + 0.033*"walk" + 0.025*"stand" + 0.024*"park" + 0.024*"girl" + 0.022*"line" + 0.021*"security" + 0.017*"drop" + 0.017*"seat"'),
 (4,
  '0.051*"arab" + 0.035*"landlord" + 0.032*"arabic" + 0.025*"difference" + 0.023*"raise" + 0.023*"chicken" + 0.018*"saudi" + 0.018*"shawarma" + 0.015*"fake" + 0.015*"bear"'),
 (5,
  '0.044*"notice" + 0.033*"quality" + 0.030*"scam" + 0.026*"clean" + 0.024*"minimum" + 0.023*"poor" + 0.022*"complain" + 0.0

In [98]:
#dataset of topcis and topic representation
num_topics = model_5.num_topics

topics_words = []

for topic in range(num_topics):
    topic_words = model_5.show_topic(topic, topn = 10)
    words = [word[0] for word in topic_words]
    topics_words.append({"topic": topic, "words": words})
    

#create a dataframe
model_5_topics_df = pd.DataFrame(topics_words)
model_5_topics_df.set_index('topic', inplace =True)

with option_context('display.max_colwidth', None):
    display(model_5_topics_df)
    


,words
topic,
0,"[report, passport, travel, process, emirate, email, contract, wear, cancel, court]"
1,"[owner, head, turn, hotel, room, light, lmao, bother, curious, fresh]"
2,"[hand, face, sense, hold, common, literally, public, short, stupid, absolutely]"
3,"[away, date, walk, stand, park, girl, line, security, drop, seat]"
4,"[arab, landlord, arabic, difference, raise, chicken, saudi, shawarma, fake, bear]"
5,"[notice, quality, scam, clean, minimum, poor, complain, wage, super, apple]"
6,"[care, accept, allow, respect, rule, muslim, treat, follow, force, okay]"
7,"[speak, house, deal, remember, agent, estate, english, lose, afford, villa]"
8,"[rent, area, apartment, cheap, expensive, building, depend, canada, plan, trip]"


<ul>
    <li>topic 0: label--><em><b>immigration_travel</b></em></li><br>
    <li>topic 1:terms not belonging to a single coherent theme or related themes</li><br>
    <li>topic 2: terms not belonging to a single coherent theme or related themes</li><br>
    <li>topic 3: label --><em><b>social_activities</b></em></li><br>
    <li>topic 4: </li><br>
    <li>topic 5: </li><br>
    <li>topic 6: label --><em><b>culture_norms</b></em></li><br>
    <li>topic 7: five of the terms(<em>'house', 'deal', 'agent', 'estate', 'villa'</em>) belong to label --><em><b>real_estate_housing</b></em></li><br>
    <li>topic 8: a repeat time of topic 7, with terms like <em> 'rent', 'area', 'apartment', 'cheap', 'expensive', 'building'</em> belonging to label --><em><b>real_estate_housing</b></em></li><br>
    <li>topic 9: <em> 'bank', 'account', 'charge', 'transaction'</em>, label --><em><b>financial_services</b></em></li><br>
    <li>topic 10: </li><br>
    <li>topic 11: topic 1: Label --> <em><b>social_relationships</b></em></li><br>
    <li>topic 12: a jumble of terms, with some that can be labelled <em><b>health.covid</b></em></li><br>
    <li>topic 13: </li><br>
    <li>topic 14: label--><em><b>communication_media</b></em> **not of interest**</li><br>
    <li>topic 15: label--><em><b>urban_mobility.traffic_regulations</b></em></li><br>
    <li>topic 16: a jumble of terms, relating to different themes</li><br>
    <li>topic 17: label--><em><b>food_dining_services</b></em></li><br>
    <li>topic 18: do not relate to themes considered</li>
    <li>topic 19: label--><em><b>employment_opportunities</b></em></li><br>
    <li>topic 20: a jumble of terms belong to unrelated themes</li><br>
    <li>topic 21: some terms like <em> 'insurance', 'car', 'license', 'center', 'cover'</em> that relate to label--><em><b>urban_mobility.car_ownership</b></em></li><br>
    <li>topic 22: label--><em><b>shopping_services</b></em></li><br>
    <li>topic 23: a jumble of terms unrelated to themes considered</li><br>
    <li>topic 24: label --><em><b>international_communities_inclusion</b></em></li><br>
   
    
        
</ul>